In [0]:
%sql
USE CATALOG dbacademy;

In [0]:
# %sql
# CREATE SCHEMA IF NOT EXISTS 1_bronze_db;
# CREATE SCHEMA IF NOT EXISTS 2_silver_db;
# CREATE SCHEMA IF NOT EXISTS 3_gold_db;

In [0]:
spark.sql("""
          SELECT * FROM read_files(
              '/Volumes/dbacademy/ops/ops_data/orders/00.json',
              format => 'json',
              schema => 'order_id int, customer_id int, order_date date, amount float, items array<string>'
          )
          """).show()

+--------+-----------+----------+------+--------------------+
|order_id|customer_id|order_date|amount|               items|
+--------+-----------+----------+------+--------------------+
|    1001|         40|2024-11-28| 60.51|[Finally, Offer, ...|
|    1002|         34|2025-05-16|403.06|[Establish, Techn...|
|    1003|         10|2025-07-08|294.56| [Risk, Information]|
|    1004|         14|2025-05-06|416.29|              [Last]|
|    1005|         41|2024-12-07|307.63|[Owner, Rest, Sec...|
|    1006|         24|2025-06-02| 85.21|[Interesting, Mov...|
|    1007|         29|2025-02-11|361.17|[Interesting, Lan...|
|    1008|         17|2024-12-04|387.06|              [Task]|
|    1009|         45|2025-02-12|250.23|  [Common, Not, Ago]|
|    1010|          9|2024-10-29|385.54|[Face, High, Prep...|
|    1011|         37|2024-12-09|216.79|              [Dark]|
|    1012|         36|2025-03-25|245.84|       [Worry, Make]|
|    1013|          5|2025-03-23|348.59|[White, Trip, Old...|
|    101

# **Bronze**
> JSON file -> bronze layer

In [0]:
%sql
CREATE OR REPLACE TABLE default.orders_bronze
AS SELECT *, current_timestamp() AS added_time 
FROM read_files(
  '/Volumes/dbacademy/ops/ops_data/orders/00.json',
  format => 'json',
  schema => 'order_id int, customer_id int, order_date date, amount float, items array<string>'
)

num_affected_rows,num_inserted_rows


# **Silver**
> bronze layer -> silver layer

In [0]:
%sql
CREATE OR REPLACE TABLE default.orders_silver AS 
SELECT order_id, order_date, amount FROM default.orders_bronze

num_affected_rows,num_inserted_rows


# **Gold**

In [0]:
%sql
CREATE OR REPLACE TABLE default.orders_gold AS
SELECT order_date, SUM(amount) AS total_amount, COUNT(*) AS total_sales FROM default.orders_silver
GROUP BY order_date
ORDER BY SUM(amount)

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM default.orders_silver;

order_id,order_date,amount
1001,2024-11-28,60.51
1002,2025-05-16,403.06
1003,2025-07-08,294.56
1004,2025-05-06,416.29
1005,2024-12-07,307.63
1006,2025-06-02,85.21
1007,2025-02-11,361.17
1008,2024-12-04,387.06
1009,2025-02-12,250.23
1010,2024-10-29,385.54


In [0]:
%sql
SELECT * FROM default.orders_gold;

order_date,total_amount,total_sales
2025-05-15,23.31999969482422,1
2025-01-04,37.43000030517578,1
2025-04-16,43.84000015258789,1
2025-03-29,53.72999954223633,1
2024-12-17,66.86999893188477,2
2024-09-16,72.41000366210938,1
2025-03-30,74.37000274658203,1
2024-08-13,79.33000183105469,1
2025-03-07,80.51000213623047,1
2025-06-03,81.2300033569336,1
